In [1]:
import sys
sys.path.append("../..")

In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from models.src import train_model

2024-04-12 04:42:56.860986: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-12 04:42:57.736350: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/opt/conda/lib
2024-04-12 04:42:57.736465: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl

In [4]:
from models.src import evaluate, utils

In [11]:
model_list = ["GaussianNB","HistGradientBoostingClassifier",
                "AdaBoostClassifier",
                "RandomForestClassifier",
                "KNeighborsClassifier",
                "MLPClassifier",
                "SVC",
                "DecisionTreeClassifier"]
param_grid = [{},
             {'max_leaf_nodes': [10, 20, 31], },
             {'n_estimators': [10, 50, 100], 'algorithm': ['SAMME']},
             {"max_leaf_nodes": [None, 20, 10], "n_estimators": [50, 100]},
             {"n_neighbors": [3, 5,10,20]},
             {'hidden_layer_sizes':[[30,]]},
             {"kernel": ['rbf'], "C": [ 1]},
             {}]

In [12]:
outputs = []

In [13]:
for i in tqdm(range(len(model_list))):
    
    model_config = {"model_type": 'traditional', "clf": model_list[i], 
                   "name": f"{model_list[i]}_30sec"}
    data_config = {"use_file": "../../data/features_30_sec.csv", # so no preprocess
                   "split_shuffle": True,
                   "split_stratify": True,
                   "train_split": 0.7, 
                   "test_split": 0.15,
                   "seed": 2024, }
    train_config = {
                    "model_type": "traditional",
                    "grid_search": param_grid[i], #'max_leaf_nodes': [30, 60, 120]},
                    "n_folds": 5, 
                    "metric": ['f1_macro', 'accuracy'], 
                    "seed": 2024,
                    "save_location": "../output"
                    }
    outputs.append(train_model.train(model_config, data_config, train_config,))

 75%|███████▌  | 6/8 [00:23<00:05,  2.63s/it]/opt/conda/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [14]:
test_results = pd.DataFrame([outputs[i].results for i in range(len(model_list))])
test_results['model'] = model_list
test_results = test_results.iloc[:, np.r_[4, 0:4]].sort_values(by = "accuracy", ascending = False)

In [44]:
test_results.to_csv("../output/test_results.csv")

In [15]:
test_results

,model,accuracy,f1-macro,worst-performing-label,best-performing-label
1,HistGradientBoostingClassifier,0.826667,0.827316,rock,classical
3,RandomForestClassifier,0.800000,0.794943,reggae,classical
7,DecisionTreeClassifier,0.660000,0.655382,rock,classical
0,GaussianNB,0.440000,0.404916,country,classical
2,AdaBoostClassifier,0.426667,0.422347,pop,metal
6,SVC,0.286667,0.226399,blues,classical
4,KNeighborsClassifier,0.280000,0.268255,rock,classical
5,MLPClassifier,0.240000,0.222826,country,classical


In [16]:
cv_results = pd.DataFrame([outputs[i].search.cv_results_ for i in range(len(model_list))])
cv_results['model'] = model_list

In [19]:
# cv_results = utils.explode(cv_results, list(set(cv_results.columns)-{'model'}))

In [20]:
# cv_results